In [ ]:
import pandas as pd
import numpy as np
import skimage.io as sk
from skimage import img_as_ubyte
from skimage.io import imread
from scipy import spatial
from tensorflow.keras.layers import Dense, Flatten, Input, Lambda, MaxPooling2D, Conv2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from glob import glob
from PIL import Image
import cv2
import matplotlib.pyplot as plt

In [ ]:
train_path = 'sign_data/train'
test_path = 'sign_data/test'

In [ ]:
Image_Width = 512
Image_Height = 512
Image_Size = (Image_Width, Image_Height)
Image_Channel = 3
batch_size=15

In [ ]:
import os
import tensorflow as tf
import numpy as np

# Image settings and batch size
Image_Width, Image_Height, Image_Channel = 224, 224, 3
batch_size = 32

def get_image_paths_and_labels(base_dir):
    image_paths = []
    labels = []
    
    for user_folder in os.listdir(base_dir):
        user_path = os.path.join(base_dir, user_folder)
        if not os.path.isdir(user_path):
            continue
        
        # Determine label: 1 for forged, 0 for genuine
        if "_forg" in user_folder:
            label = 1
        else:
            label = 0
        
        # Collect image paths
        for img_name in os.listdir(user_path):
            img_path = os.path.join(user_path, img_name)
            if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(img_path)
                labels.append(label)
    
    return image_paths, labels

# Get training and testing image paths and labels
train_image_paths, train_labels = get_image_paths_and_labels("sign_data/train")
test_image_paths, test_labels = get_image_paths_and_labels("sign_data/test")

# Shuffle the dataset (optional but useful)
train_data = list(zip(train_image_paths, train_labels))
test_data = list(zip(test_image_paths, test_labels))
np.random.shuffle(train_data)
np.random.shuffle(test_data)
train_image_paths, train_labels = zip(*train_data)
test_image_paths, test_labels = zip(*test_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

def make_pairs(image_paths, labels):
    """
    Create positive and negative pairs.
    For each image, we create:
      - A positive pair: another image with the same label.
      - A negative pair: an image with a different label.
    """
    pairs = []
    pair_labels = []
    
    # Map each label to the list of image paths with that label
    label_to_paths = {}
    for path, label in zip(image_paths, labels):
        label_to_paths.setdefault(label, []).append(path)
    
    # Create pairs
    for i in range(len(image_paths)):
        current_path = image_paths[i]
        current_label = labels[i]
        
        # Positive pair (if possible, choose a different image from the same class)
        if len(label_to_paths[current_label]) > 1:
            possible_pos = [p for p in label_to_paths[current_label] if p != current_path]
            pos_path = np.random.choice(possible_pos)
            pairs.append((current_path, pos_path))
            pair_labels.append(1)
        
        # Negative pair: choose an image from a different label
        negative_labels = [lbl for lbl in label_to_paths.keys() if lbl != current_label]
        if negative_labels:
            neg_label = np.random.choice(negative_labels)
            neg_path = np.random.choice(label_to_paths[neg_label])
            pairs.append((current_path, neg_path))
            pair_labels.append(0)
            
    return pairs, np.array(pair_labels)

# Create image pairs and corresponding labels for training and testing
train_pairs, train_pair_labels = make_pairs(train_image_paths, train_labels)
test_pairs, test_pair_labels = make_pairs(test_image_paths, test_labels)

def preprocess_pair(image_pair, label):
    # Unstack the tensor along the first axis to get the two file paths.
    img1_path, img2_path = tf.unstack(image_pair, num=2, axis=0)
    
    # Preprocess the first image
    img1 = tf.io.read_file(img1_path)
    img1 = tf.image.decode_jpeg(img1, channels=3)
    img1 = tf.image.resize(img1, [Image_Width, Image_Height])
    img1 = img1 / 255.0  # Normalize

    # Preprocess the second image
    img2 = tf.io.read_file(img2_path)
    img2 = tf.image.decode_jpeg(img2, channels=3)
    img2 = tf.image.resize(img2, [Image_Width, Image_Height])
    img2 = img2 / 255.0  # Normalize

    return (img1, img2), label

# Create TensorFlow datasets from the pairs
train_ds = tf.data.Dataset.from_tensor_slices((train_pairs, train_pair_labels))
train_ds = train_ds.map(preprocess_pair, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_pairs, test_pair_labels))
test_ds = test_ds.map(preprocess_pair, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Now you can train your Siamese network with:
# siamese_model.fit(train_ds, validation_data=test_ds, epochs=40)


In [ ]:
model = Sequential()

## Conv layer 1
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(Image_Width,Image_Height, Image_Channel)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

## Conv layer 2
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

## Conv layer 3
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

## Conv layer 4
model.add(Conv2D(256, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

## Conv layer 5
model.add(Conv2D(256, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

## Conv layer 6
model.add(Conv2D(512, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_ds, validation_data=test_ds, epochs=40)

In [ ]:
from tensorflow.keras.models import load_model

model.save('forge_2.h5')

In [ ]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load the trained model
model = tf.keras.models.load_model("forge_2.h5")

# Compile the model (if not compiled)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Image settings
Image_Width, Image_Height = 224, 224
test_dir = "sign_data/test"

def preprocess_image(image_path):
    """Preprocesses an image for model inference."""
    image = Image.open(image_path).convert("RGB")
    image = image.resize((Image_Width, Image_Height))
    image = img_to_array(image) / 255.0  # Normalize
    return np.expand_dims(image, axis=0)

# Prepare test dataset
test_images = []
test_labels = []

for user_folder in os.listdir(test_dir):
    user_path = os.path.join(test_dir, user_folder)
    if not os.path.isdir(user_path):
        continue

    label = 1 if "_forg" in user_folder else 0  # 0 = Forgery, 1 = Genuine
    
    for img_name in os.listdir(user_path):
        img_path = os.path.join(user_path, img_name)
        if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            test_images.append(preprocess_image(img_path))
            test_labels.append(label)


test_images = np.vstack(test_images)
test_labels = np.array(test_labels)

predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)  


accuracy = accuracy_score(test_labels, predicted_labels)
report = classification_report(test_labels, predicted_labels, target_names=["Forgery", "Genuine"])
conf_matrix = confusion_matrix(test_labels, predicted_labels)


print(f"Test Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

# For binary classification with softmax, we assume predictions is computed from model.predict(test_images)
# and test_labels are the ground-truth labels.
# We use the probability for class 1.
probabilities = predictions[:, 1]

# Define thresholds from 0 to 1 (e.g., 0, 0.01, ..., 1.0)
thresholds = np.linspace(0, 1, 101)

# Lists to store the computed metrics for each threshold
accuracies = []
f1_scores = []
precisions = []
recalls = []
fprs = []  # False Positive Rates

# Iterate over each threshold value
for threshold in thresholds:
    # Convert probabilities to binary predictions based on the current threshold
    predicted_labels_threshold = (probabilities >= threshold).astype(int)
    
    # Calculate metrics
    acc = accuracy_score(test_labels, predicted_labels_threshold)
    f1 = f1_score(test_labels, predicted_labels_threshold)
    prec = precision_score(test_labels, predicted_labels_threshold)
    rec = recall_score(test_labels, predicted_labels_threshold)
    
    # Compute confusion matrix and extract true negatives (TN) and false positives (FP)
    tn, fp, fn, tp = confusion_matrix(test_labels, predicted_labels_threshold).ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    
    # Store metrics for later plotting
    accuracies.append(acc)
    f1_scores.append(f1)
    precisions.append(prec)
    recalls.append(rec)
    fprs.append(fpr)
    
    # Print the metrics for the current threshold
    print(f"Threshold: {threshold:.2f} | Accuracy: {acc:.4f} | F1: {f1:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | FPR: {fpr:.4f}")

# Plot all metrics versus the confidence threshold
plt.figure(figsize=(10, 6))
plt.plot(thresholds, accuracies, label='Accuracy', marker='o')
plt.plot(thresholds, f1_scores, label='F1 Score', marker='o')
plt.plot(thresholds, precisions, label='Precision', marker='o')
plt.plot(thresholds, recalls, label='Recall', marker='o')
plt.plot(thresholds, fprs, label='False Positive Rate', marker='o')
plt.xlabel("Confidence Threshold")
plt.ylabel("Metric Value")
plt.title("Metrics vs. Confidence Threshold")
plt.legend()
plt.grid(True)
plt.show()
